# Federated Statistics with image data

In this example, we will compute local and global image statistics with the consideration that data is private at each of the client sites.

## Define target statistics configuration

For Image statistics, we are only interested in histogram of the image intensity, so we ignore all other statistic measures. 

```
statistic_configs = {"count": {}, "histogram": {"*": {"bins": 20, "range": [0, 256]}}}
```

## Define the local statistics generator

Based on the above target statistics configuration, we can define the local statistics generator. To do this, we need to write a class that implement 

```

class Statistics(InitFinalComponent, ABC):

    def initialize(self, fl_ctx: FLContext):
    def pre_run(self, statistics: List[str], num_of_bins: Optional[Dict[str, Optional[int]]],bin_ranges: Optional[Dict[str, Optional[List[float]]]]):
    def features(self) -> Dict[str, List[Feature]]:
    def count(self, dataset_name: str, feature_name: str) -> int:
    def sum(self, dataset_name: str, feature_name: str) -> float:
    def mean(self, dataset_name: str, feature_name: str) -> float:
    def stddev(self, dataset_name: str, feature_name: str) -> float:
    def variance_with_mean(self, dataset_name: str, feature_name: str, global_mean: float, global_count: float) -> float:
    def histogram(self, dataset_name: str, feature_name: str, num_of_bins: int, global_min_value: float, global_max_value: float) -> Histogram:
    def max_value(self, dataset_name: str, feature_name: str) -> float:
    def min_value(self, dataset_name: str, feature_name: str) -> float:
    def failure_count(self, dataset_name: str, feature_name: str) -> int:
    def quantiles(self, dataset_name: str, feature_name: str, percentiles: List) -> Dict:
    def finalize(self, fl_ctx: FLContext):

```

But since we are only interested in two metrics : Count and Histogram, we can ignore other metrics implementation and only implements count and histogram. Here is the skeleton code for this generator

```

class ImageStatistics(Statistics):

    def __init__(self):
        pass
 
    def initialize(self, fl_ctx: FLContext):
        self.fl_ctx = fl_ctx
        self.client_name = fl_ctx.get_identity_name()
        
        # call load data function 

    def _load_data_list(self, client_name, fl_ctx: FLContext) -> bool:
        pass


    def pre_run(
        self,
        statistics: List[str],
        num_of_bins: Optional[Dict[str, Optional[int]]],
        bin_ranges: Optional[Dict[str, Optional[List[float]]]],
    ):
        return {}

    def features(self) -> Dict[str, List[Feature]]:
        return {"train": [Feature("intensity", DataType.FLOAT)]}

    def count(self, dataset_name: str, feature_name: str) -> int:

        # return number of images loaded
        pass
            

    def failure_count(self, dataset_name: str, feature_name: str) -> int:

        return self.failure_images

    def histogram(
        self, dataset_name: str, feature_name: str, num_of_bins: int, global_min_value: float, global_max_value: float
    ) -> Histogram:
        # do histogram calculation: 
        return Histogram(HistogramType.STANDARD, histogram_bins)
```

Here ```FLContext``` is the context of the current Job workflow, "identity" referring the site identity, therefor: ```get_identity_name()``` will return trhe site name.


You can take a look of the implementation

In [ ]:
! cat code/src/image_statistics.py

# Define Job Configuration
 

```
    statistic_configs = {"count": {}, "histogram": {"*": {"bins": 20, "range": [0, 256]}}}
    
    # define local stats generator
    stats_generator = ImageStatistics(data_root_dir)

    job = StatsJob(
        job_name="stats_image",
        statistic_configs=statistic_configs,
        stats_generator=stats_generator,
        output_path=output_path,
    )
```

## Install requirements

In [2]:
%pip install -r code/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 31.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 31.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 MB 31.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 32.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 29.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 MB 30.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 31.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 30.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 28.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 

## Download data

As an example, we use the dataset from the ["COVID-19 Radiography Database"](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database).
it contains png image files in four different classes: `COVID`, `Lung_Opacity`, `Normal`, and `Viral Pneumonia`.
First create a temp directory, then we download and extract to `/tmp/nvflare/image_stats/data/.`.




In [ ]:
! pip install kagglehub

In [4]:
%%bash 

# prepare the directory

if [ ! -d /tmp/nvflare/image_stats/data ]; then
  mkdir -p /tmp/nvflare/image_stats/data
fi


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tawsifurrahman/covid19-radiography-database")

print("Path to dataset files:", path)



Download and unzip the data (you may need to log in to Kaggle or use an API key). Once you have extracted the data from the zip file, you can check the directory to make sure you have the COVID-19_Radiography_Dataset directory at the following location.

In [ ]:
! mv {path} /tmp/nvflare/image_stats/data/

! tree /tmp/nvflare/image_stats/data


## Prepare data

Next, create the data lists simulating different clients with varying amounts and types of images. 
The downloaded archive contains subfolders for four different classes: `COVID`, `Lung_Opacity`, `Normal`, and `Viral Pneumonia`.
Here we assume each class of image corresponds to a different site.

In [ ]:
! code/data/prepare_data.sh

## Run Job with FL Simulator

The file [image_stats_job.py](code/image_stats_job.py) uses `StatsJob` to generate a job configuration in a Pythonic way. With the default arguments, the job will be exported to `/tmp/nvflare/jobs/image_stats` and then the job will be run with the FL simulator with the `simulator_run()` command with a work_dir of `/tmp/nvflare/workspace/image_stats`.

In [4]:
%cd code

! python3 image_stats_job.py

%cd -


/home/chester/projects/NVFlare/examples/tutorials/self-paced-training/part-1_federated_learning_introduction/chapter-2_develop_federated_learning_applications/02.1_federated_statistics/federated_statistics_with_image_data/code
2025-02-21 20:21:56,135 - StatisticsController - INFO - fed_stats control flow started.
2025-02-21 20:21:56,136 - StatisticsController - INFO - start prepare inputs for task fed_stats_1st_statistics
2025-02-21 20:21:56,136 - StatisticsController - INFO - task: fed_stats statistics_flow for fed_stats_1st_statistics started.
2025-02-21 20:22:03,115 - StatisticsTaskHandler - INFO - Executing task 'fed_stats' for client: 'site-2'
2025-02-21 20:22:03,115 - StatisticsPrivacyFilter - INFO - start StatisticsPrivacyFilter
2025-02-21 20:22:03,116 - StatisticsPrivacyFilter - INFO - apply StatisticPrivacyFilter for client site-2
2025-02-21 20:22:03,116 - MinCountCleanser - INFO - apply MinCountCheck for client site-2
2025-02-21 20:22:03,116 - AddNoiseToMinMax - INFO - AddNoi

## Examine the result


The results are stored on the server in the workspace at "/tmp/nvflare/image_stats" and can be accessed with the following command:

In [6]:
! ls -al /tmp/nvflare/workspace/image_stats/server/simulate_job/statistics/image_stats.json
         

-rw-rw-r-- 1 chester chester 2260 Feb 21 20:22 /tmp/nvflare/workspace/image_stats/server/simulate_job/statistics/image_stats.json


## Visualization
We can visualize the results easly via the visualization notebook. Before we do that, we need to copy the data to the notebook directory 


In [9]:
! cp /tmp/nvflare/workspace/image_stats/server/simulate_job/statistics/image_stats.json code/image_stats/demo/.

now we can visualize via the [visualization notebook](code/image_stats/demo/visualization.ipynb)

## We are done !
Congratulations, you just completed the federated stats image histogram calulation
